<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 
Author: [Yury Kashnitskiy](https://yorko.github.io) (@yorko). Edited by Sergey Kolchenko (@KolchenkoSergey). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.

## <center>Assignment #6
### <center> Beating baselines in "How good is your Medium article?"
    
<img src='../../img/medium_claps.jpg' width=40% />


[Competition](https://www.kaggle.com/c/how-good-is-your-medium-article). The task is to beat "A6 baseline" (~1.45 Public LB score). Do not forget about our shared ["primitive" baseline](https://www.kaggle.com/kashnitsky/ridge-countvectorizer-baseline) - you'll find something valuable there.

**Your task:**
 1. "Freeride". Come up with good features to beat the baseline "A6 baseline" (for now, public LB is only considered)
 2. You need to name your [team](https://www.kaggle.com/c/how-good-is-your-medium-article/team) (out of 1 person) in full accordance with the [course rating](https://drive.google.com/open?id=19AGEhUQUol6_kNLKSzBsjcGUU3qWy3BNUg8x8IFkO3Q). You can think of it as a part of the assignment. 16 credits for beating the mentioned baseline and correct team naming.
 
*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel #mlcourse_ai, pinned thread __#a6__*

In [2]:
import os
import json
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_absolute_error
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import Ridge

The following code will help to throw away all HTML tags from an article content.

In [3]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

Supplementary function to read a JSON line without crashing on escape characters.

In [4]:
def read_json_line(line=None):
    result = None
    try:        
        result = json.loads(line)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(str(e).split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        new_line = list(line)
        new_line[idx_to_replace] = ' '
        new_line = ''.join(new_line)     
        return read_json_line(line=new_line)
    return result

Extract features `content`, `published`, `title` and `author`, write them to separate files for train and test sets.

In [21]:
def extract_features_and_write(path_to_data,
                               inp_filename, is_train=True):
    
    features = ['content', 'published', 'title', 'author']
    prefix = 'train' if is_train else 'test'
    feature_files = [open(os.path.join(path_to_data,
                                       '{}_{}.txt'.format(prefix, feat)),
                          'w', encoding='utf-8')
                     for feat in features]
    
    features_to_files_dict = dict(list(zip(features, feature_files)))
    to_dump = {'content': [], 'published': [], 'title': [], 'author': []}
    
    with open(os.path.join(path_to_data, inp_filename), 
              encoding='utf-8') as inp_json_file:

        for line in tqdm_notebook(inp_json_file):
            json_data = read_json_line(line)
            pub_date = json_data['published']['$date']
            title = json_data['meta_tags']['title'].split('\u2013')[0].strip().replace('\n', ' ').replace('\r', ' '))
            author_name = json_data['meta_tags']['author'].strip()
            content = strip_tags(json_data['content'].replace('\n', ' ').replace('\r', ' '))
            to_dump['content'].append(content)
            to_dump['published'].append(pub_date)
            to_dump['title'].append(title)
            to_dump['author'].append(author_name)   
            for feature in features_to_files_dict:
                features_to_files_dict[feature].write(str(to_dump[feature][-1]) + '\n')
            #print(pub_date,"|", title, "|", author_name, "|", content)

        for f in features_to_files_dict:
            features_to_files_dict[f].close()
    return to_dump
                

In [22]:
PATH_TO_DATA = '../../data/medium/' # modify this if you need to

In [23]:
with open(os.path.join(PATH_TO_DATA, "train.json"), 
          encoding='utf-8') as inp_json_file:
    i = 0
    for line in inp_json_file: #tqdm_notebook(inp_json_file):
        if i > 20:
            break
        json_data = read_json_line(line)
        pub_date = json_data['published']['$date']
        title = json_data['title'].replace('\n', ' ').replace('\r', ' ')
#         author_name = json_data['author']['name'].replace('\n', ' ').replace('\r', ' ')
        content = strip_tags(json_data['content'].replace('\n', ' ').replace('\r', ' '))
        pub_date = np.datetime64(json_data['published']['$date'])
        title = json_data['meta_tags']['title'].split('\u2013')[0].strip().replace('\n', ' ').replace('\r', ' '))
        author_name = json_data['meta_tags']['author'].strip()
        print(pub_date,"||", title, "||", author_name)#, "|", content)
#         print(json.dumps(json_data, indent=4))
        i += 1
       

2012-08-13T22:54:53.510 || Medium Terms of Service || Medium
2015-08-03T07:44:50.331 || Amendment to Medium Terms of Service Applicable to U.S. Government Users || Medium
2017-02-05T13:08:17.410 || 走入山與海之間：閩東大刀會和兩岸走私 || Yun-Chen Chien（簡韻真）
2017-05-06T08:16:30.776 || How fast can a camera get? || Vaibhav Khulbe
2017-06-04T14:46:25.772 || A game for the lonely fox || Vaibhav Khulbe
2017-04-02T16:21:15.171 || Now That’s Comedy || Kate Reed Petty
2016-08-15T04:16:02.103 || Che fine ha fatto «Gola Profonda»? || exedre
2015-01-14T21:31:07.568 || Airbnb Guerilla Usability Testing || Raghav Haran
2014-02-11T04:11:54.771 || A Guerilla Usability Test on Dropbox Photos || Francine Lee
2015-10-25T02:58:05.551 || How to Get Any Job You Want (even if you’re unqualified) || Raghav Haran
2016-08-15T15:31:13.601 || On Trump, Brexit and Predictions; || E²
2016-08-09T21:01:06.303 || 29 Books Every Entrepreneur Should Read in 2016 || One Month
2016-09-08T15:47:57.336 || The Basics of Staking in Poker || F

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  if sys.path[0] == '':


In [25]:
train_data = extract_features_and_write(PATH_TO_DATA, 'train.json', is_train=True)

In [29]:
test_data = extract_features_and_write(PATH_TO_DATA, 'test.json', is_train=False)

In [12]:
! ls -l ../../data/medium/


total 4148732
-rw-rw-r-- 1 1000 1000     884217 Nov 17 16:56 sample_submission.csv
-rw-rw-rw- 1 1000 1000 1156020029 Sep 20 06:56 test.json
-rw-r--r-- 1 root root     173225 Nov 17 19:20 test_author.txt
-rw-r--r-- 1 root root  347316676 Nov 17 19:20 test_content.txt
-rw-r--r-- 1 root root     831480 Nov 17 19:20 test_published.txt
-rw-r--r-- 1 root root    2350112 Nov 17 19:20 test_title.txt
-rw-rw-rw- 1 1000 1000 2086185062 Sep 20 06:56 train.json
-rw-r--r-- 1 root root     895415 Nov 18 07:18 train_author.txt
-rw-r--r-- 1 root root  648268497 Nov 18 07:18 train_content.txt
-rw-rw-r-- 1 1000 1000     912544 Nov 17 16:56 train_log1p_recommends.csv
-rw-r--r-- 1 root root    1495512 Nov 18 07:18 train_published.txt
-rw-r--r-- 1 root root    2924954 Nov 18 07:18 train_title.txt


**Add the following groups of features:**
    - Tf-Idf with article content (ngram_range=(1, 2), max_features=100000 but you can try adding more)
    - Tf-Idf with article titles (ngram_range=(1, 2), max_features=100000 but you can try adding more)
    - Time features: publication hour, whether it's morning, day, night, whether it's a weekend
    - Bag of authors (i.e. One-Hot-Encoded author names)

In [30]:
train = pd.DataFrame()
train['published'] = pd.to_datetime(train_data['published'], format='%Y-%m-%dT%H:%M:%S.%fZ')
train['title'] = train_data['title']
train['author'] = train_data['author']
train['content'] = train_data['content']

In [37]:
idx_split = len(train)

In [ ]:
df_full['dow'] = df_full['published'].apply(lambda x: x.dayofweek)
df_full['year'] = df_full['published'].apply(lambda x: x.year)
df_full['month'] = df_full['published'].apply(lambda x: x.month)
df_full['hour'] = df_full['published'].apply(lambda x: x.hour)

full_new_feat['start_hour'] = full_df['time1'].\
    apply(lambda ts:ts.hour).astype('float64')
full_new_feat.head(5)
full_new_feat['morning'] = (full_new_feat['start_hour'] <=11.0).astype('int64')

In [ ]:
TF_V = TfidfVectorizer(max_features=100000, ngram_range=(1,2))


In [39]:
X_train_content_sparse = TF_V.fit_transform(train_data['content'])

In [ ]:
X_train_title_sparse = TF_V.fit_transform(train_data['title'])

**Join all sparse matrices.**

In [ ]:
X_train_sparse = hstack([X_train_content_sparse, X_train_title_sparse,
                         X_train_author_sparse, 
                         X_train_time_features_sparse]).tocsr()

In [ ]:
X_test_sparse = hstack([X_test_content_sparse, X_test_title_sparse,
                        X_test_author_sparse, 
                        X_test_time_features_sparse]).tocsr()

**Read train target and split data for validation.**

In [ ]:
train_target = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_log1p_recommends.csv'), 
                           index_col='id')
y_train = train_target['log_recommends'].values

In [ ]:
train_part_size = int(0.7 * train_target.shape[0])
X_train_part_sparse = X_train_sparse[:train_part_size, :]
y_train_part = y_train[:train_part_size]
X_valid_sparse =  X_train_sparse[train_part_size:, :]
y_valid = y_train[train_part_size:]

**Train a simple Ridge model and check MAE on the validation set.**

In [ ]:
# You code here

**Train the same Ridge with all available data, make predictions for the test set and form a submission file.**

In [ ]:
# You code here

In [ ]:
def write_submission_file(prediction, filename,
                          path_to_sample=os.path.join(PATH_TO_DATA, 
                                                      'sample_submission.csv')):
    submission = pd.read_csv(path_to_sample, index_col='id')
    
    submission['log_recommends'] = prediction
    submission.to_csv(filename)

In [ ]:
write_submission_file(ridge_test_pred, os.path.join(PATH_TO_DATA,
                                                    'assignment6_medium_submission.csv'))

**Now's the time for dirty Kaggle hacks. Form a submission file with all zeros. Make a submission. What do you get if you think about it? How is it going to help you with modifying your predictions?**

In [ ]:
write_submission_file(np.zeros_like(ridge_test_pred), 
                      os.path.join(PATH_TO_DATA,
                                   'medium_all_zeros_submission.csv'))

**Modify predictions in an appropriate way (based on your all-zero submission) and make a new submission.**

In [ ]:
ridge_test_pred_modif = ridge_test_pred # You code here

In [ ]:
write_submission_file(ridge_test_pred_modif, 
                      os.path.join(PATH_TO_DATA,
                                   'assignment6_medium_submission_with_hack.csv'))

That's it for the assignment. Much more credits will be given to the winners in this competition, check [course roadmap](https://mlcourse.ai/roadmap). Do not spoil the assignment and the competition - don't share high-performing kernels (with MAE < 1.5).

Some ideas for improvement:

- Engineer good features, this is the key to success. Some simple features will be based on publication time, authors, content length and so on
- You may not ignore HTML and extract some features from there
- You'd better experiment with your validation scheme. You should see a correlation between your local improvements and LB score
- Try TF-IDF, ngrams, Word2Vec and GloVe embeddings
- Try various NLP techniques like stemming and lemmatization
- Tune hyperparameters. In our example, we've left only 50k features and used C=1 as a regularization parameter, this can be changed
- SGD and Vowpal Wabbit will learn much faster
- Play around with blending and/or stacking. An intro is given in [this Kernel](https://www.kaggle.com/kashnitsky/ridge-and-lightgbm-simple-blending) by @yorko 
- In our course, we don't cover neural nets. But it's not obliged to use GRUs/LSTMs/whatever in this competition.

Good luck!

<img src='../../img/kaggle_shakeup.png' width=50%>